In [1]:
# --- Import libraries ---
import re, shutil
from pathlib import Path

# --- 1) กำหนดโฟลเดอร์ root ---
root = Path(r"CD:\GitHub\dolap\files_dolab\vfiles")  # ใช้ r"" เพื่อรองรับ backslash
ext = "py"   # กรองเฉพาะไฟล์ .py
quarantine_mode = True   # True = ย้ายไป quarantine, False = ลบจริง

# --- 2) แพทเทิร์นตรวจจับไฟล์แปลกปลอม ---
PATTERNS = [
    r"\bimport\s+cx_Oracle\b",
    r"init_oracle_client\s*\(",
    r"\bcx_Oracle\.connect\s*\(",
    r"\bdsn\s*=",
    r"\busername\s*=\s*['\"].+?['\"]",
    r"\bpassword\s*=\s*['\"].+?['\"]",
    r"\b(ip|host)\s*=\s*['\"]\d{1,3}(?:\.\d{1,3}){3}['\"]",  # IPv4 literal
    r"\bport\s*=\s*['\"]?\d{2,5}['\"]?",
    r"\b(database|service_name)\s*=\s*['\"].+?['\"]",
    r"\bmakedsn\s*\(",
    r"[\w\.\-]+:\d{2,5}/[A-Za-z0-9_\-\.]+",  # รูปแบบ dsn เช่น 10.54.54.179:1521/xe
]
COMPILED = [re.compile(p, re.IGNORECASE | re.DOTALL) for p in PATTERNS]

def is_suspicious(text: str) -> bool:
    return any(p.search(text) for p in COMPILED)

# --- 3) สแกนไฟล์ ---
suspects = []
for p in root.rglob(f"*.{ext}"):
    try:
        txt = p.read_text(encoding="utf-8", errors="ignore")
    except Exception:
        continue
    if is_suspicious(txt):
        suspects.append(p)

print(f"[INFO] พบไฟล์ต้องสงสัย {len(suspects)} ไฟล์")
for i, p in enumerate(suspects, 1):
    print(f"  {i:>3}. {p}")

# --- 4) จัดการไฟล์ ---
if suspects:
    if quarantine_mode:
        qdir = root / "__quarantine__"
        qdir.mkdir(exist_ok=True)
        for p in suspects:
            target = qdir / p.name
            suffix = 1
            while target.exists():
                target = qdir / f"{p.stem}__{suffix}{p.suffix}"
                suffix += 1
            shutil.move(str(p), str(target))
        print(f"[DONE] ย้ายไฟล์ {len(suspects)} ไฟล์ ไปยัง {qdir}")
    else:
        for p in suspects:
            if p.exists():
                p.unlink()
        print(f"[DONE] ลบไฟล์ {len(suspects)} ไฟล์ เรียบร้อย")

[INFO] พบไฟล์ต้องสงสัย 0 ไฟล์
